In [1]:
#include <iostream>
#include <string>
#include <vector>
#include <algoviz/SVG.hpp>
#include <iomanip>
#include <algoviz/HTML.hpp>
using namespace std;

In [2]:
//Einbeziehen der Klassen der Figuren

#include "BauerWeiss.hpp"
#include "BauerSchwarz.hpp"
#include "Laufer.hpp"
#include "Dame.hpp"
#include "Koenig.hpp"
#include "Pferd.hpp"
#include "Turm.hpp"

In [3]:
//Variablen

AlgoViz::clear();


// Array mit der aktuellen Besetzung des Schachfeldes
int Feld[64] = {-1,-2,-3,-4,-5,-3,-2,-1,
                -6,-6,-6,-6,-6,-6,-6,-6,
                 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0,
                 0, 0, 0, 0, 0, 0, 0, 0,
                 6, 6, 6, 6, 6, 6, 6, 6,
                 1, 2, 3, 4, 5, 3, 2, 1};

MouseState maus;
MouseState maus2;

// Bewegung 
bool selected = false;
bool seccond_click = false;

// Der aktuelle Ort
double x = 0;
double y = 0;

// Der Zielort
int zielX = 0;
int zielY = 0;

int zielFeldX = 0;
int zielFeldY = 0;

// Die Richtung der Geschwindigkeit
double vx;
double vy;

// Der Betrag der Geschwindigkeit
int speed = 30;

Rect selectedRect;

Image* letzteFigur = NULL;

//Spielfeld
SVG view = SVG(400,420,"Schach");
//SVG text = SVG(400,400,"Protokoll");

Image bild = Image("Bilder/Spielfeld.png",200,200,400,400,&view);
//view.setStatusMsg("Drücken Sie zum Beenden eine beliebige Taste!");

string Spieler1 = "Johannes";
string Spieler2 = "Jonas";

Text t;

vector<Image> toteFiguren;
vector<int> toteSpieler;
vector<Image*> friedhof;
vector<Rect> moglicheRect;
vector<Image*> BauernEingetauscht;
vector<Image*> neueDamen;
Image *kopie;
Image *BauerEingetauscht;
vector<int> Züge;
Image * Back = new Image("Bilder/Pfeil.png", 350,410 ,20,20,&view);


In [4]:
// Funktion zur Beschriftung des Randes des Schachfeldes 

void printFeld( int SFeld[64]){
    cout<< "   A  B  C  D  E  F  G  H";
    
    // Schleife zur Numerierung 
    for (int i = 0; i<64;i++){
        if(i % 8 == 0 ){
            cout<<endl << i / 8 + 1;
        }
        cout<< setw(3) << SFeld[i];

    }
    cout << endl;
}

In [5]:
// Berechnung der Koordinate von Feldern

float FeldToKor(float Feld) {
    return ( Feld - 1 ) * 46.3 + 16 + 22 ; 
}

In [6]:
// Funktion zur Berechnung von Koordinate zum Feld

int KorToFeld(int Kor) {
    return (Kor - 16) / 46 + 1;
}

In [7]:
// Funktion zur Umrandung von Feldern

Rect drawSelect(int x, int y, string Farbe = "blue"){
    //Nimmt des Feld, um das ein Rechteck gezeichnet werden soll
    Rect r = Rect( FeldToKor(x)-22, FeldToKor(y)-22, 46, 46, &view);
    r.setColor(Farbe);
    r.setFill("transparent");
    r.setStrokeWidth(5);
    return r;
}

In [8]:
// Berechet aus einem String der Position (A1) 
// den zugehörigen int Wert im Array (A1=0)

int SchachFeldToFeld(string s){
  
    int x;
    int y;
    y = (s[1])-'0';
    x = s[0] - 'A';
    
    return (x + (y-1) *8);    
}

In [9]:
// Einführen & Einfügen der Bilder der Spielfiguren

// schwarze Figuren
vector<Image*> FigurenS;

for(int i = 1; i < 9; i++){
    FigurenS.push_back(new Image("Bilder/Bauer(schwarz).png",FeldToKor(i),FeldToKor(7),40,40,&view));
}
    
FigurenS.push_back(new Image("Bilder/Konig(schwarz).png",FeldToKor(5),FeldToKor(8),40,40,&view));
FigurenS.push_back(new Image("Bilder/Dame(schwarz).png",FeldToKor(4),FeldToKor(8),40,40,&view));
FigurenS.push_back(new Image("Bilder/Laufer(schwarz).png",FeldToKor(3),FeldToKor(8),40,40,&view));
FigurenS.push_back(new Image("Bilder/Laufer(schwarz).png",FeldToKor(6),FeldToKor(8),40,40,&view));
FigurenS.push_back(new Image("Bilder/Turm(schwarz).png",FeldToKor(1),FeldToKor(8),40,40,&view));
FigurenS.push_back(new Image("Bilder/Turm(schwarz).png",FeldToKor(8),FeldToKor(8),40,40,&view));
FigurenS.push_back(new Image("Bilder/Springer(schwarz).png",FeldToKor(2),FeldToKor(8),40,40,&view));
FigurenS.push_back(new Image("Bilder/Springer(schwarz).png",FeldToKor(7),FeldToKor(8),40,40,&view));


// weiße Figuren
vector<Image*> FigurenW;

for(int i = 1; i < 9; i++){
    FigurenW.push_back(new Image("Bilder/Bauer(weiß).png",FeldToKor(i),FeldToKor(2),40,40,&view));
}

FigurenW.push_back(new Image("Bilder/Konig(weiß).png",FeldToKor(5),FeldToKor(1),40,40,&view));
FigurenW.push_back(new Image("Bilder/Dame(weiß).png",FeldToKor(4),FeldToKor(1),40,40,&view));
FigurenW.push_back(new Image("Bilder/Laufer(weiß).png",FeldToKor(3),FeldToKor(1),40,40,&view));
FigurenW.push_back(new Image("Bilder/Laufer(weiß).png",FeldToKor(6),FeldToKor(1),40,40,&view));
FigurenW.push_back(new Image("Bilder/Turm(weiß).png",FeldToKor(1),FeldToKor(1),40,40,&view));
FigurenW.push_back(new Image("Bilder/Turm(weiß).png",FeldToKor(8),FeldToKor(1),40,40,&view));
FigurenW.push_back(new Image("Bilder/Springer(weiß).png",FeldToKor(2),FeldToKor(1),40,40,&view));
FigurenW.push_back(new Image("Bilder/Springer(weiß).png",FeldToKor(7),FeldToKor(1),40,40,&view));

In [10]:
// Funktion zur Anzeige der Züge 
// (Beispiel: "schwarzer Bauer auf E5")

void anzeigeDesZuges(int neuePosition, string altePositionString, string neuePositionString, int spielfeld[64]){

    int Spieler = spielfeld[neuePosition];
    string Figur;
    
    // Suchen der passenden Spielfigur auf dem ausgewählten Feld
    
    //schwarz
    if(Spieler == 6){
       Figur = "schwarzer Bauer"; 
    } else if(Spieler == 5){
       Figur = "schwarzer König"; 
    } else if(Spieler == 4){
       Figur = "schwarze Dame"; 
    } else if(Spieler == 3){
       Figur = "schwarzer Läufer"; 
    } else if(Spieler == 2){
       Figur = "schwarzes Pferd"; 
    } else if(Spieler == 1){
       Figur = "schwarzer Turm"; 
    }
    
    //weiß
    else if(Spieler == -6){
       Figur = "weißer Bauer"; 
    } else if(Spieler == -5){
       Figur = "weißer König"; 
    } else if(Spieler == -4){
       Figur = "weiße Dame"; 
    } else if(Spieler == -3){
       Figur = "weißer Läufer"; 
    } else if(Spieler == -2){
       Figur = "weißes Pferd"; 
    } else if(Spieler == -1){
       Figur = "weißer Turm";   
    } else {
        Figur = ""; 
    }
    
    cout<<Figur<<" von "<<altePositionString<<" auf "<< neuePositionString<<endl<<endl;
    t = Text(Figur + " von " + altePositionString + " auf " + neuePositionString , 40, 420,&view);
}

In [11]:
// Berechnet aus einer int x-Koordinate den richtigen Buchstaben

string FeldzuBuchstabe(int x){
    char Feld = 'A'-1 + x;
    string s;
    s = Feld;
    return s;
}

In [12]:
// Rückgabe vollständige Position in einem String (zB. E5)

string FeldToSchachFeld(int Feld){
    
    string s = "";
    int x;
    int y;
    y = Feld / 8 +1;
    x = Feld % 8 + 1;
    s = FeldzuBuchstabe(x) + to_string(y);
   
    return s;    
}

In [13]:
// Rückgabe vollständige Position in zwei Zahlen ((A1=11)

string FeldToSchachFeldZahlen(int Feld){
    
    string s = "";
    int x;
    int y;
    y = Feld / 8 +1;
    x = Feld % 8 + 1;
    s =  to_string(x)+ to_string(y);
  
    return s;   
}

In [14]:
// Entfernt Figuren vom Friedhof (zB wenn man zurück drückt)

void friedhofremove(vector<Image*> &friedhof){
    
    friedhof.at(friedhof.size()-1)->removeFromView();
    friedhof.erase(friedhof.begin() + friedhof.size()-1);
    int größe = friedhof.size();
    if ( größe % 8 == 0 ){
        view.setViewBox(0,0,400 + (größe / 8) * 46 ,420,"Schach");
        
    }
}

In [15]:
// Geschlagene Spieler werden neben dem Spielfeld angezeigt #Friedhof

void friedhofadd(int toteSpieler, vector<Image*> &friedhof){
    
    //Spieler zum Friedhof hinzufügen
    int größe = friedhof.size();
    int y;
    float x;
    
    //View erweitern
    if ( größe % 8 == 0 ){
        view.setViewBox(0,0,455 + (größe / 8) * 46 ,420,"Schach");
        
    }
    x = 9.5 + größe / 8;
    y = (größe % 8) + 1;
    
    // Feldabfrage des geschlagenen Feldes & Überprüfung welche Figur war.
    switch (toteSpieler){
        
        // schwarz
        case 6:
            friedhof.push_back(new Image("Bilder/Bauer(schwarz).png",FeldToKor(x),FeldToKor(y),40,40,&view));
            break;
        case 5: 
            friedhof.push_back(new Image("Bilder/Konig(schwarz).png",FeldToKor(x),FeldToKor(y),40,40,&view));
            break;     
        case 4:    
            friedhof.push_back(new Image("Bilder/Dame(schwarz).png",FeldToKor(x),FeldToKor(y),40,40,&view));
            break;
        case 3:    
            friedhof.push_back(new Image("Bilder/Laufer(schwarz).png",FeldToKor(x),FeldToKor(y),40,40,&view));   
            break;
        case 2:    
            friedhof.push_back(new Image("Bilder/Springer(schwarz).png",FeldToKor(x),FeldToKor(y),40,40,&view));
            break;
        case 1:    
            friedhof.push_back(new Image("Bilder/Turm(schwarz).png",FeldToKor(x),FeldToKor(y),40,40,&view));
            break;
        
        // weiß
        case -6:
            friedhof.push_back(new Image("Bilder/Bauer(weiß).png",FeldToKor(x),FeldToKor(y),40,40,&view));
            break;
        case -5: 
            friedhof.push_back(new Image("Bilder/Konig(weiß).png",FeldToKor(x),FeldToKor(y),40,40,&view));
            break;     
        case -4:    
            friedhof.push_back(new Image("Bilder/Dame(weiß).png",FeldToKor(x),FeldToKor(y),40,40,&view));
            break;
        case -3:    
            friedhof.push_back(new Image("Bilder/Laufer(weiß).png",FeldToKor(x),FeldToKor(y),40,40,&view));   
            break;
        case -2:    
            friedhof.push_back(new Image("Bilder/Springer(weiß).png",FeldToKor(x),FeldToKor(y),40,40,&view));
            break;
        case -1:    
            friedhof.push_back(new Image("Bilder/Turm(weiß).png",FeldToKor(x),FeldToKor(y),40,40,&view));
            break;
        default:
            break;
    }
}

In [16]:
// Animation auf dem Schachfeld bei gegebenem Anlass xD

void animation(int Anzahl){
    
    AlgoViz::html("<audio autoplay src=Music/audio2.mp3 type= audio/mp3 </audio>");
    
    srand (time(NULL));
    string eineFarbe;
    vector<string> Farben = {"blue", "red", "green", "yellow", "lightblue", "black", "white"};
    
    for(int i = 0; i < Anzahl; i++) {
        
        eineFarbe = Farben[i % Farben.size()];
        int x = rand() % 8 + 1;
        int y = rand() % 8 + 1;
        Rect r = drawSelect(x, y, eineFarbe);
        AlgoViz::sleep(50);
        r.removeFromView();    
    }
}

In [17]:
// zweite Animation auf dem Schachfeld bei gegebenem Anlass xD

void animation2(int Anzahl){
    
    AlgoViz::html("<audio autoplay src=Music/audio1.mp3 type= audio/mp3 </audio>");
    srand (time(NULL));
    string eineFarbe;
    vector<string> Farben = {"blue", "red", "green", "yellow", "lightblue", "black", "white"};
    
    for(int i = 0; i < Anzahl; i++) {
        
        
        for(int j = 1; j <= 8; j++){
            eineFarbe = Farben[j % Farben.size()];
            Rect r1 = drawSelect(j, j, eineFarbe);
            Rect r2 = drawSelect(9-j, j, eineFarbe);
            Rect r3 = drawSelect(j, 9-j, eineFarbe);
            Rect r4 = drawSelect(9-j, 9-j, eineFarbe);
            AlgoViz::sleep(50);
            r1.removeFromView();
            r2.removeFromView();
            r3.removeFromView();
            r4.removeFromView();
        }
    }
}

In [18]:
// dritte Animation auf dem Schachfeld bei gegebenem Anlass xD

void animation3(int Anzahl){
    AlgoViz::html("<audio autoplay src=Music/sieg.mp3 type= audio/mp3 </audio>");
    
    srand (time(NULL));
    string eineFarbe;
    vector<string> Farben = {"blue", "red", "green", "yellow", "lightblue", "black", "white"};
    
    for(int i = 0; i < Anzahl; i++) {
       
        for(int j = 1; j <= 8; j++){
            
            eineFarbe = Farben[j % Farben.size()];
            Rect r1 = drawSelect(j, 1, eineFarbe);
            Rect r2 = drawSelect(1, j, eineFarbe);
            Rect r3 = drawSelect(j, 8, eineFarbe);
            Rect r4 = drawSelect(8, j, eineFarbe);
            AlgoViz::sleep(200);
            r1.removeFromView();
            r2.removeFromView();
            r3.removeFromView();
            r4.removeFromView();
        }
    }
}

In [19]:
// vierte Animation auf dem Schachfeld bei gegebenem Anlass xD

void animation4(int Anzahl){
    
    //Music
    AlgoViz::html("<audio autoplay src=Music/start2.mp3 type= audio/mp3 </audio>");
    
    srand (time(NULL));
    string eineFarbe;
    vector<string> Farben = {"blue", "red", "green", "yellow", "lightblue", "black", "white"};
    for(int i = 0; i < Anzahl; i++) {
        //int y = rand() % 8 + 1;
        for(int j = 2; j <= 8; j++){
            eineFarbe = Farben[j % Farben.size()];
            Rect r1 = drawSelect(j, 1, eineFarbe);
            Rect r2 = drawSelect(j-1, 1, eineFarbe);
            AlgoViz::sleep(10);
            r1.removeFromView();
            r2.removeFromView();
        }
        for(int j = 2; j <= 8; j++){
            eineFarbe = Farben[j % Farben.size()];
            Rect r1 = drawSelect(8, j, eineFarbe);
            Rect r2 = drawSelect(8, j-1, eineFarbe);
            AlgoViz::sleep(10);
            r1.removeFromView();
            r2.removeFromView();
        }
        for(int j = 2; j <= 8; j++){
            eineFarbe = Farben[j % Farben.size()];
            Rect r1 = drawSelect(9-j, 8, eineFarbe);
            Rect r2 = drawSelect(10-j, 8, eineFarbe);
            AlgoViz::sleep(10);
            r1.removeFromView();
            r2.removeFromView();
        }
        for(int j = 2; j <= 8; j++){
            eineFarbe = Farben[j % Farben.size()];
            Rect r1 = drawSelect(1, 10-j, eineFarbe);
            Rect r2 = drawSelect(1, 9-j, eineFarbe);
            AlgoViz::sleep(10);
            r1.removeFromView();
            r2.removeFromView();
        }
  
    }
}

In [20]:
// Funktion des Schlagens
// geschlagene Figur wird vom Spielfeld entfernt

void schlagen(vector<Image*> &Gegner, int x, int y, vector<Image> &dead, vector<int> &Zügee, vector<int> &toteSpieler, int SpielerName, vector<Image*> &friedhof, int spielfeld[64]){
    
    //x und y sind Felder also z.B. 8/8
    //Falls sich an der Pos ein Gegner befindet, wird dieser entfert.
    
    //int SpielerName = spielfeld[Spieler];
    string Figur;
    //cout<<SpielerName<<endl;
    // Suchen der passenden Spielfigur auf dem ausgewählten Feld
    if(SpielerName == 6){
       Figur = "schwarzer Bauer"; 
    } else if(SpielerName == 5){
       Figur = "schwarzer König"; 
    } else if(SpielerName == 4){
       Figur = "schwarze Dame"; 
    } else if(SpielerName == 3){
       Figur = "schwarzer Läufer"; 
    } else if(SpielerName == 2){
       Figur = "schwarzes Pferd"; 
    } else if(SpielerName == 1){
       Figur = "schwarzer Turm"; 
    } else if(SpielerName == -6){
       Figur = "weißer Bauer"; 
    } else if(SpielerName == -5){
       Figur = "weißer König"; 
    } else if(SpielerName == -4){
       Figur = "weiße Dame"; 
    } else if(SpielerName == -3){
       Figur = "weißer Läufer"; 
    } else if(SpielerName == -2){
       Figur = "weißes Pferd"; 
    } else if(SpielerName == -1){
       Figur = "weißer Turm";   
    } else {
        Figur = "leer"; 
    }
    
    for( int i = 0; i < Gegner.size(); i++){
        
        if( KorToFeld(Gegner[i]->getX()) == x & KorToFeld(Gegner[i]->getY()) == y){
            
            //Element zu tote (nicht Poionter!!)
            dead.push_back(*Gegner[i]);
            
            //Kopie verstecken
            dead[dead.size()-1].hide();
            
            //Nachricht für Kill
            Zügee.push_back(999);
            
            //Um in der Matrix Später die richtige Figur reinzumachen:)
            toteSpieler.push_back(SpielerName);
            
            //Friedhof akktuallisieren
            friedhofadd(SpielerName, friedhof);
            
            Gegner[i]->removeFromView();
            
            //auch aus Vektor löschen:
            Gegner.erase(Gegner.begin() + i);
            
            //Nach entfernen muss figur irgenwo gespeichert werden
            cout<< endl << Figur << " geschlagen auf: " << FeldzuBuchstabe(x) << y << endl;
            animation2(2);
            return;
        }    
    }    
}

In [21]:
// zwei Int-Werte zu einem Int-Wert des Schachfeld Arrays (11=0)

int FeldToArray(int x, int y){
    return (x-1 + 8 * ( y - 1 ));
}

In [22]:
void addZug(vector<int> &v, int xZiel, int yZiel, int x, int y){
    //Fügt den gemachten Zug unserem Vector mit allen Zügen hinzu
    v.push_back(FeldToArray(x,y));
    v.push_back(FeldToArray(xZiel,yZiel));
}

In [23]:
void MatrixAktrualisieren(int altesFeld, int neuesFeld, int Feld[], bool killed, vector<int> &toteSpieler){
    //Nach einem Zug, Muss unser Aktuelles Spielfeld Aktrualisiert werden.
    Feld[neuesFeld] = Feld[altesFeld];
    Feld[altesFeld] = 0;
    
    if(killed){
        Feld[altesFeld] = toteSpieler.at(toteSpieler.size()-1);
        toteSpieler.pop_back();
    }
}

In [24]:
Image* figurSuchen(vector<Image*> Figuren, int xFeld, int yFeld){
    //Gibt die Figur zurück, die auf einem bestimmten Feld steht -> Break umgehen
    for( Image *Figur : Figuren ){
        if( KorToFeld(Figur->getX()) == xFeld & KorToFeld(Figur->getY()) == yFeld){
            return Figur;
        }
    }
    
    return NULL;
}

In [25]:
// Funktion zum Annulieren einer Rochade 
// (bei Klich auf zurück-Taste)

void RochadeBack(int spielfeld[64], vector<Image*> FigurenW, vector<Image*> FigurenS, vector<int> &Züge){
    Image* Turm;
    
    //Rochade weiß - links oben
    if(Züge[Züge.size()-1] == 510){
        spielfeld[3] = 0;
        spielfeld[0] = -1;
        Züge.pop_back();
        Turm = figurSuchen(FigurenW,4,1);
        Turm->moveTo(FeldToKor(1),FeldToKor(1));
    }
    
    //Rochade weiß - rechts oben
    if(Züge[Züge.size()-1] == 150){
        spielfeld[5] = 0;
        spielfeld[7] = -1;
        Züge.pop_back();
        Turm = figurSuchen(FigurenW,6,1);
        Turm->moveTo(FeldToKor(8),FeldToKor(1));
    }
    
    //Rochade schwarz - links unten
    if(Züge[Züge.size()-1] == 2510){
        spielfeld[59] = 0;
        spielfeld[56] = 1;
        Züge.pop_back();
        Turm = figurSuchen(FigurenS,4,8);
        Turm->moveTo(FeldToKor(1),FeldToKor(8));
    }
    
    //Rochade schwarz - rechts unten    
    if(Züge[Züge.size()-1] == 2150){
        spielfeld[61] = 0;
        spielfeld[63] = 1;
        Züge.pop_back();
        Turm = figurSuchen(FigurenS,6,8);
        Turm->moveTo(FeldToKor(8),FeldToKor(8));
    }
}

In [26]:
// Durchführen des Sonderzuges Rochade
// in diesem Zug wird nicht nur eine Figur (König) bewegt, sondern auch eine zweite (Turm)

void ausführenRochade(int position, int neuePosition, int spielfeld[64], vector<Image*> Figuren, vector<int> &Züge){
     
     Image* Turm;
    
     //weißer König - Rochade links
     if (position == 4 && neuePosition == 2 && spielfeld[2] == -5 && spielfeld[0]== -1){
        
         Turm = figurSuchen(Figuren,1,1);
         Turm->moveTo(FeldToKor(4),FeldToKor(1));
         spielfeld[61] = 0;
         spielfeld[63] = -1;
         Züge.push_back(510); //Spezial code
         animation3(2);
         return;
     }  
    
    //weißer König - Rochade rechts
     if (position == 4 && neuePosition == 6 && spielfeld[6] == -5 && spielfeld[7]== -1){
        
         Turm = figurSuchen(Figuren,8,1);
         Turm->moveTo(FeldToKor(6),FeldToKor(1));
         spielfeld[7] = 0;
         spielfeld[5] = -1;
         Züge.push_back(150); //Spezial code
         animation3(2);
         return;
     }  
    
    //schwarzer König - Rochade links
     if (position == 60 && neuePosition == 58 && spielfeld[58] == 5 && spielfeld[56]== 1){
         
         Turm = figurSuchen(Figuren,1,8);
         Turm->moveTo(FeldToKor(4),FeldToKor(8));
         spielfeld[56] = 0;
         spielfeld[59] = 1;
         Züge.push_back(2510); //Spezial code
         animation2(2);
         return;
     }  
    
    //schwarzer König - Rochade rechts
     if (position == 60 && neuePosition == 62 && spielfeld[62] == 5 && spielfeld[63]== 1){
    
         Turm = figurSuchen(Figuren,8,8);
         Turm->moveTo(FeldToKor(6),FeldToKor(8));
         spielfeld[63] = 0;
         spielfeld[61] = 1;
         Züge.push_back(2150); //Spezial code
         animation2(2);
         return;
     }  
   
}    

In [27]:
// Funktion zur Anzeige der möglichen Züge

auto moeglichevonFigur(int feld_im_array, int Feld[64]){
    
    // Erstellung von Objekten der Spielfigurenklassen
    BauerSchwarz b = BauerSchwarz();
    BauerWeiss bw = BauerWeiss();
    Koenig k = Koenig();
    Dame d = Dame();
    Pferd p = Pferd();
    Turm t = Turm();
    Laufer l = Laufer();
    
    // Funktionsweise der Figuren ist bis auf Bauern nicht farbabhängig.
    vector<int> moeglichkeiten;
    int Spieler = Feld[feld_im_array];
    
    if(Spieler != 6 & Spieler != -6 ){
        Spieler = abs(Spieler);
    }
    
    // Suchen der passenden Spielfigur auf dem ausgewählten Feld
    switch (Spieler){
        //schwarz
        case 6:
            return b.moeglicheZuege(feld_im_array, Feld);
            break;
        case 5:
            return k.moeglicheZuege(feld_im_array, Feld);
            break;
        case 4:    
            return d.moeglicheZuege(feld_im_array, Feld); 
            break;
        case 3:
            
            return l.moeglicheZuege(feld_im_array, Feld);    
        case 2:    
            return p.moeglicheZuege(feld_im_array, Feld);
            break;
        case 1:    
            return t.moeglicheZuege(feld_im_array, Feld);  
            break;  
        case -6:
            return bw.moeglicheZuege(feld_im_array, Feld);
            break;
        default:
            return moeglichkeiten;
    }    
}

In [28]:
// Anzeige der Möglichen Züge beim über Felder fahren

void darüberFahren(vector<Image*> Figuren, int xFeld, int yFeld, vector<Rect> &moglicheRect, Image* &letzteFigur, int Feld[64]){

    Rect moeglichkeitRect;
    Image* AktuelleFigur;
    AktuelleFigur = figurSuchen(Figuren, xFeld, yFeld);
    
    
    
    if( AktuelleFigur != letzteFigur ){
        moglicheRect.clear();
    } 
    if( AktuelleFigur != NULL & AktuelleFigur != letzteFigur ){
        
        vector<int> moegl = moeglichevonFigur( FeldToArray(xFeld, yFeld), Feld );
        for( int i : moegl ){
            
            moeglichkeitRect = drawSelect( 1 + (i % 8), i / 8 + 1 , "green");
            moglicheRect.push_back(moeglichkeitRect);  
            
        }  
        
        letzteFigur = AktuelleFigur;
    }    
}

In [29]:
// Überprüft Spielende (Spielende = der gegnerische König wurde geschlagen)

bool ende(int feld[64]){
    
    bool koenig_tot_weiß = true;
    bool koenig_tot_schwarz = true;
    
    // Durchlaufen der Felder auf der Suche nach den Königen
    for( int i = 0; i < 64; i++){
        if( feld[i] == -5 ){
            koenig_tot_weiß = false;
        }
        
        if( feld[i] == 5 ){
            koenig_tot_schwarz = false;
        }
    }
    
    if( koenig_tot_weiß ){      
        cout << "Schwarz hat gewonnen!"<<endl;
        animation3(10);
        return true;
    }
    
    if( koenig_tot_schwarz ){
        cout << "Weiß hat gewonnen!"<<endl;
        animation3(10);
        return true;
    }
    
    //Wenn beide noch Leben
    return false;
    
}

In [30]:
Image* BauerEingetauscht;

In [31]:
int FeldToKor(int Feld) {
    return ( Feld - 1 ) * 46.3 + 16 + 22 ; 
}

In [32]:
// Funktion zum Sonderzug Bauer eitauschen

void bauer_zu_dame(int feld[64], vector<Image*> &Figuren, int zielFeldX, int zielFeldY, vector<int> &Züge, int bewegteFigur){

    //Code für Dame bekommen
    Züge.push_back(646);
    
    BauerEingetauscht =  figurSuchen(Figuren, zielFeldX, zielFeldY );
    BauernEingetauscht.push_back(BauerEingetauscht);
    
    if(BauerEingetauscht != NULL){
            BauerEingetauscht->hide();
    }

    bool find = false;
    
    for(int i = 0; i< Figuren.size() & find ==false; i++){
       
        if(Figuren[i] == BauerEingetauscht){
            Figuren.erase(Figuren.begin() + i);
            find = true;
        }
    }

    // weiß
    if( bewegteFigur == -6 ){
        //Dame
        neueDamen.push_back(new Image("Bilder/Dame(weiß).png",FeldToKor(zielFeldX),FeldToKor(zielFeldY),40,40,&view));
        Figuren.push_back(neueDamen[neueDamen.size()-1]);
        feld[zielFeldX - 1 + (zielFeldY -1) * 8] = -4;
    }
    
    // schwarz
    if( bewegteFigur == 6 ){
        //Dame
        neueDamen.push_back(new Image("Bilder/Dame(schwarz).png",FeldToKor(zielFeldX),FeldToKor(zielFeldY),40,40,&view));
        Figuren.push_back(neueDamen[neueDamen.size()-1]);
        feld[zielFeldX - 1 + (zielFeldY -1) * 8] = 4;
    }
    
    animation(50);    
}

In [33]:
bool check_eigenerSpieler(int Spieler, vector<Image*> FigurenW, vector<Image*> FigurenS, int ZielX, int ZielY){

    if( Spieler < 0 ){
        for(Image* i : FigurenW){
            if(KorToFeld(i->getX()) == ZielX & KorToFeld(i->getY()) == ZielY){
                return true;
            }
        }     
    } else{
        for(Image* i : FigurenS){
            if( KorToFeld(i->getX()) == ZielX & KorToFeld(i->getY()) == ZielY ){
                return true;
            }
        }        
    }
    
    return false;   
}    

In [34]:
void solo(int Feld[64], vector<Image*> FigurenS, int &zielFeldX, int &zielFeldY, Image* &SelectedFigur){
    bool gultig = false;
    srand (time(NULL));
    
    while(!gultig){
        int x = rand() % FigurenS.size();
        SelectedFigur = FigurenS[x];
        //cout<<endl<<x<<endl;
        
        vector<int> m = moeglichevonFigur( KorToFeld(SelectedFigur->getX()) -1 + (KorToFeld(SelectedFigur->getY())-1) * 8  , Feld);
        
        if( m.size() != 0 ){
            
            int z = m[rand() % m.size()];
            zielFeldX = 1 + (z % 8);
            //cout<<endl<<zielFeldX<<endl;
            zielFeldY =  z / 8 + 1; 
            //cout<<endl<<zielFeldX<<endl;
            gultig = true;
        }
    }
    
    
    
}

In [35]:
cout<< "Wer ist Spieler 1 (der darf anfangen:)?"<<endl;
do{
    cin>> Spieler1;
}while(Spieler1 == "");

cout<< "Wer ist Spieler 2?"<<endl;
do{
    cin>> Spieler2;
}while(Spieler2 == "");

Wer ist Spieler 1 (der darf anfangen:)?
sdfa
Wer ist Spieler 2?
afds


In [36]:
//Hauptprogramm
int neuesFeldSpieler;
Image* SelectedFigur;
bool soloSpiel = false; //Für Spiel gegen Bot, achtung Zug zurück geht nicht
selected = false;
view.setStatusMsg("Drücken Sie zum Beenden eine beliebige Taste!");
bool weiß = true; //Weiß darf beginnen, dann schwarz
MouseState mausBewegung;
bool schach = false;
string taste = "";

animation4(3);

while ((taste == "") && (schach == false)) {

    //Für Bot (nur wenn zuvor soloSpiel auf true gesetzte wurde)
    if (!weiß && soloSpiel == true) {
        //int Feld[64], vector<Image*> FigurenS, int &zielFeldX, int &zielFeldY, Image* &SelectedFigur)
        solo(Feld, FigurenS, zielFeldX, zielFeldY, SelectedFigur);
        selected = true;
        seccond_click = true;
        zielX = FeldToKor(zielFeldX);
        zielY = FeldToKor(zielFeldY);
        double dist = sqrt((zielX - SelectedFigur->getX()) * (zielX - SelectedFigur->getX()) + (zielY - SelectedFigur->getY()) * (zielY - SelectedFigur->getY()));

        vx = speed * (zielX - SelectedFigur->getX()) / dist;
        vy = speed * (zielY - SelectedFigur->getY()) / dist;

        neuesFeldSpieler = Feld[zielFeldX - 1 + (zielFeldY - 1) * 8];
        Feld[zielFeldX - 1 + (zielFeldY - 1) * 8] = Feld[(KorToFeld(SelectedFigur->getX()) - 1) + (KorToFeld(SelectedFigur->getY()) - 1) * 8];
        Feld[(KorToFeld(SelectedFigur->getX()) - 1) + (KorToFeld(SelectedFigur->getY()) - 1) * 8] = 0;

        addZug(Züge, zielFeldX, zielFeldY, KorToFeld(SelectedFigur->getX()), KorToFeld(SelectedFigur->getY()));
        weiß = 1 - weiß;
        schlagen(FigurenW, zielFeldX, zielFeldY, toteFiguren, Züge, toteSpieler, neuesFeldSpieler, friedhof, Feld);
    } //Bot ende
    
    
    //Mouse Input
    MouseState maus;
    MouseState maus2;

    taste = view.lastKey();
    if (!selected) {
        maus = view.lastClick();
        //Um Fehler zu vermeiden, sodass ich etwas abwählen kann
    }
    else {
        maus2 = view.lastClick();
        //Um Fehler zu vermeiden, sodass ich etwas abwählen kann
    }

    //Um über Figuren zu fahren
    mausBewegung = view.getMouseState();

    
    
    
    //ZurückKnopf:
    if (maus.x() >= 320 & maus.x() <= 370 & maus.y() >= 390 & maus.y() <= 430 & Züge.size() > 0) {
        //Wenn Zug zurück geklickt wurde, muss überprüft werden, 
        //ob letzter Zug Rochade, Bauer zu Damen Tausch oder eine Figur geschlagen wurde, 
        //dafür gibt es Speziel Codes
        
        
        //war der letzte zug eine Rochade?
        RochadeBack(Feld, FigurenW, FigurenS, Züge);
        
        //Zulest neue Dame geholt?        
        bool damegetauscht = false;
        if (Züge.at(Züge.size() - 1) == 646) {
            //cout<<"33333"<<endl;
            damegetauscht = true;
            Züge.pop_back();
        }


        //wurde zuletzt jemand geschlagen?
        bool killed = false;
        if (Züge.at(Züge.size() - 1) == 999) {

            killed = true;
            Züge.pop_back();
        }
        
        //Jetzt wird die Figur an die Letzte Stelle gesetzt
        int letztesFeld = Züge.at(Züge.size() - 1);
        int vorletztesFeld = Züge.at(Züge.size() - 2);

        //Liefert strings, mit - 0 zu int
        int XFeldVomVorletzen = FeldToSchachFeldZahlen(vorletztesFeld)[0] - '0';
        int YFeldVomVorletzen = FeldToSchachFeldZahlen(vorletztesFeld)[1] - '0';
        Züge.pop_back();
        Züge.pop_back();

        bool gefunden = false;
        
        //Jetzt wird nach der Figur gesucht, die zurück muss
        for (int i = 0; i < FigurenS.size() & gefunden != true; i++) {

            if (KorToFeld(FigurenS[i]->getX()) - 1 + 8 * (KorToFeld(FigurenS[i]->getY()) - 1) == letztesFeld) {

                FigurenS[i]->moveTo(FeldToKor(XFeldVomVorletzen), FeldToKor(YFeldVomVorletzen));

                gefunden = true;
            }
        }

        if (gefunden == false) {

            for (int i = 0; i < FigurenW.size() & gefunden != true; i++) {

                if (KorToFeld(FigurenW[i]->getX()) - 1 + 8 * (KorToFeld(FigurenW[i]->getY()) - 1) == letztesFeld) {

                    FigurenW[i]->moveTo(FeldToKor(XFeldVomVorletzen), FeldToKor(YFeldVomVorletzen));

                    gefunden = true;
                }
            }
        }
        
        //Anderer Spieler ist wieder
        if (gefunden == true) {
            weiß = 1 - weiß;

            gefunden = false;

            if (weiß) {

                //Die Figuren der anderen Farbe in schlagen Funktion, achtung: spieler hat schon gewechselt
                view.setStatusMsg(Spieler1 + " darf ziehen (weiß)");
                //wurde jemand geschlagen?
                //schlagen( FigurenW, zielFeldX, zielFeldY, toteFiguren, Züge);
            }
            else {
                view.setStatusMsg(Spieler2 + " ist dran (schwarz)");
                //schlagen( FigurenS, zielFeldX, zielFeldY, toteFiguren, Züge);
            }
        }

        //Figur die geschlagen wurde wieder erscheinen lassen

        MatrixAktrualisieren(letztesFeld, vorletztesFeld, Feld, killed, toteSpieler);
        
        
        if (damegetauscht == true) {
            BauernEingetauscht[BauernEingetauscht.size() - 1]->show();
            BauernEingetauscht[BauernEingetauscht.size() - 1]->moveTo(FeldToKor(XFeldVomVorletzen), FeldToKor(YFeldVomVorletzen));

            if (weiß) {

                FigurenS.push_back(BauernEingetauscht[BauernEingetauscht.size() - 1]);
                Feld[vorletztesFeld] = -6;
            }
            else {
                FigurenW.push_back(BauernEingetauscht[BauernEingetauscht.size() - 1]);
                Feld[vorletztesFeld] = 6;
            }

            BauernEingetauscht.pop_back();
            //Dame löschen
            neueDamen[neueDamen.size() - 1]->removeFromView();
            neueDamen.pop_back();
            //Matrix
            damegetauscht = false;
        }

        if (killed == true) {

            //Ziel hier, langes Problem, element aus Totefiguren hinten rauslöschen,
            //dieses aber davor Kopieren und zu den Figuren und zum Spielfeld wieder hinzuzufügen... locker 5 h gebraucht

            kopie = new Image(toteFiguren[toteFiguren.size() - 1]);

            if (weiß) {

                FigurenS.push_back(kopie);
                //cout<<FigurenS[FigurenS.size()-1]<<endl;
                FigurenS[FigurenS.size() - 1]->show();
            }

            else {

                FigurenW.push_back(kopie);
                //cout<<FigurenW[FigurenS.size()-1]<<endl;
                FigurenW[FigurenW.size() - 1]->show();
            }

            toteFiguren[toteFiguren.size() - 1].removeFromView();
            toteFiguren.pop_back();
            killed = false;
            //Hier wurde das elemnt wieder zerstört
            //Friedhof aktuallisiern
            friedhofremove(friedhof);
        }
    } // ZurückKnopf Ende

    
    
    
    if (selected == false) {

        if (mausBewegung.isLegal()) {

            //Beim Drüberfahren
            if (weiß) {

                darüberFahren(FigurenW, KorToFeld(mausBewegung.x()), KorToFeld(mausBewegung.y()), moglicheRect, letzteFigur, Feld);
            }
            else {

                darüberFahren(FigurenS, KorToFeld(mausBewegung.x()), KorToFeld(mausBewegung.y()), moglicheRect, letzteFigur, Feld);
            }
        }

        //Beim Klick
        if (maus.isLegal()) {
            //Bei Klick
            // Neue Zielkoordinaten
            if (maus.x() < 382 & maus.x() > 15 & maus.y() < 382 & maus.y() > 17) {
                //Welcher SPieler ist dran?
                if (weiß) {
                    SelectedFigur = figurSuchen(FigurenW, KorToFeld(maus.x()), KorToFeld(maus.y()));
                }
                else {
                    SelectedFigur = figurSuchen(FigurenS, KorToFeld(maus.x()), KorToFeld(maus.y()));
                }
                if (SelectedFigur != NULL) {
                    selected = true;
                    AlgoViz::sleep(5);
                    selectedRect = drawSelect(KorToFeld(maus.x()), KorToFeld(maus.y()));
                }
            } //Ende if maus.x()...
        }
    }

    
    if (selected == true) {

        //Nachdem eine Figur ausgewählt wurde, die Möglichkeiten ausblednen

        if (maus2.isLegal()) {

            // Neue Zielkoordinaten

            if (maus2.x() < 382 & maus2.x() > 15 & maus2.y() < 382 & maus2.y() > 17) {

                zielX = maus2.x();
                zielY = maus2.y();

                zielFeldX = (zielX - 15) / 46 + 1;
                zielFeldY = (zielY - 17) / 46 + 1;

                seccond_click = true;

                // Die Distanz zum Ziel
                double dist = sqrt((zielX - SelectedFigur->getX()) * (zielX - SelectedFigur->getX()) + (zielY - SelectedFigur->getY()) * (zielY - SelectedFigur->getY()));

                vx = speed * (zielX - SelectedFigur->getX()) / dist;
                vy = speed * (zielY - SelectedFigur->getY()) / dist;
                selectedRect.removeFromView();
                selectedRect = drawSelect(KorToFeld(maus2.x()), KorToFeld(maus2.y()));

                //wenn es ein gültiger Zug war -> anderer SPieler ist dran
                // check_eigenerSpieler(int Spieler, vector<Image*> FigurenW, vector<Image*> FigurenS, int ZielX, int ZielY)
                if (check_eigenerSpieler(Feld[KorToFeld(SelectedFigur->getX()) - 1 + (KorToFeld(SelectedFigur->getY()) - 1) * 8], FigurenW, FigurenS, zielFeldX, zielFeldY) == false) {

                    neuesFeldSpieler = Feld[zielFeldX - 1 + (zielFeldY - 1) * 8];
                    Feld[zielFeldX - 1 + (zielFeldY - 1) * 8] = Feld[(KorToFeld(SelectedFigur->getX()) - 1) + (KorToFeld(SelectedFigur->getY()) - 1) * 8];
                    Feld[(KorToFeld(SelectedFigur->getX()) - 1) + (KorToFeld(SelectedFigur->getY()) - 1) * 8] = 0;

                    //Diesen Zug hinzufügen
                    addZug(Züge, zielFeldX, zielFeldY, KorToFeld(SelectedFigur->getX()), KorToFeld(SelectedFigur->getY()));

                    //Nächster Spieler
                    weiß = 1 - weiß;

                    if (weiß) {
                        //Die Figuren der anderen Farbe in schlagen Funktion, achtung: spieler hat schon gewechselt
                        view.setStatusMsg(Spieler1 + " darf ziehen (weiß)");
                        //wurde jemand geschlagen?
                        schlagen(FigurenW, zielFeldX, zielFeldY, toteFiguren, Züge, toteSpieler, neuesFeldSpieler, friedhof, Feld);
                    }
                    else {
                        view.setStatusMsg(Spieler2 + " ist dran (schwarz)");
                        schlagen(FigurenS, zielFeldX, zielFeldY, toteFiguren, Züge, toteSpieler, neuesFeldSpieler, friedhof, Feld);
                    }
                }

                else {
                    //Zug unglültig
                    selectedRect.removeFromView();
                    seccond_click = false;
                    selected = false;
                }
            }
        }

        //wenn nochmal geklickt
        if (seccond_click == true) {

            //welche Figur?
            int bewegteFigur = Feld[zielFeldX - 1 + (zielFeldY - 1) * 8];
            int lastPos = KorToFeld(SelectedFigur->getX()) - 1 + (KorToFeld(SelectedFigur->getY()) - 1) * 8;
            int neuePos = (zielFeldX - 1 + (zielFeldY - 1) * 8);
            while (seccond_click == true) {

                // Wir prüfen, ob das Objekt noch weit genug vom Ziel entfernt ist.
                if ((abs(zielX - x) > 50) || (abs(zielY - y) > 50)) {
                    x = SelectedFigur->getX() + vx;
                    y = SelectedFigur->getY() + vy;
                    (*SelectedFigur).moveTo(x, y);
                }
                else {
                    // Ist das Objekt hinreichend nah am Ziel,
                    // setzten wir das Objekt direkt dorthin.

                    //Rochade weiß
                    if (bewegteFigur == -5) {
                        ausführenRochade(lastPos, (zielFeldX - 1 + (zielFeldY - 1) * 8), Feld, FigurenW, Züge);
                    }
                    //Rochade schwarz
                    if (bewegteFigur == 5) {
                        ausführenRochade(lastPos, (zielFeldX - 1 + (zielFeldY - 1) * 8), Feld, FigurenS, Züge);
                    }
                    x = FeldToKor(zielFeldX);
                    y = FeldToKor(zielFeldY);
                    (*SelectedFigur).moveTo(x, y);
                    selectedRect.removeFromView();

                    seccond_click = false;
                    selected = false;

                    //Prüfen ob König tot ist
                    if (ende(Feld)) {
                        schach = true;
                    }


                    anzeigeDesZuges(neuePos, FeldToSchachFeld(lastPos), FeldToSchachFeld(neuePos), Feld);

                    //DamenTauschen
                    //falls der Bauer ans Ende bewegt wird
                    if ((bewegteFigur == -6) & (zielFeldY == 8)) {
                        bauer_zu_dame(Feld, FigurenW, zielFeldX, zielFeldY, Züge, bewegteFigur);
                    }
                    if ((bewegteFigur == 6) & (zielFeldY == 1)) {
                        bauer_zu_dame(Feld, FigurenS, zielFeldX, zielFeldY, Züge, bewegteFigur);
                    }

                    //Matrix Ausgeben
                    printFeld(Feld);
                }
            }
        }
    }
}

view.setStatusMsg("Ende!");

weißer Bauer von E2 auf E3

   A  B  C  D  E  F  G  H
1 -1 -2 -3 -4 -5 -3 -2 -1
2 -6 -6 -6 -6  0 -6 -6 -6
3  0  0  0  0 -6  0  0  0
4  0  0  0  0  0  0  0  0
5  0  0  0  0  0  0  0  0
6  0  0  0  0  0  0  0  0
7  6  6  6  6  6  6  6  6
8  1  2  3  4  5  3  2  1
schwarzer Bauer von E7 auf E5

   A  B  C  D  E  F  G  H
1 -1 -2 -3 -4 -5 -3 -2 -1
2 -6 -6 -6 -6  0 -6 -6 -6
3  0  0  0  0 -6  0  0  0
4  0  0  0  0  0  0  0  0
5  0  0  0  0  6  0  0  0
6  0  0  0  0  0  0  0  0
7  6  6  6  6  0  6  6  6
8  1  2  3  4  5  3  2  1

schwarzer Läufer geschlagen auf: C8


weißer Bauer von C2 auf C8



   A  B  C  D  E  F  G  H
1 -1 -2 -3 -4 -5 -3 -2 -1
2 -6 -6  0 -6  0 -6 -6 -6
3  0  0  0  0 -6  0  0  0
4  0  0  0  0  0  0  0  0
5  0  0  0  0  6  0  0  0
6  0  0  0  0  0  0  0  0
7  6  6  6  6  0  6  6  6
8  1  2 -4  4  5  3  2  1
schwarzes Pferd von B8 auf C6

   A  B  C  D  E  F  G  H
1 -1 -2 -3 -4 -5 -3 -2 -1
2 -6 -6  0 -6  0 -6 -6 -6
3  0  0  0  0 -6  0  0  0
4  0  0  0  0  0  0  0  0
5  0  0  0  0  6  0  0  0
6  0  0  2  0  0  0  0  0
7  6  6  6  6  0  6  6  6
8  1  0 -4  4  5  3  2  1
weiße Dame von C8 auf B6

   A  B  C  D  E  F  G  H
1 -1 -2 -3 -4 -5 -3 -2 -1
2 -6 -6  0 -6  0 -6 -6 -6
3  0  0  0  0 -6  0  0  0
4  0  0  0  0  0  0  0  0
5  0  0  0  0  6  0  0  0
6  0 -4  2  0  0  0  0  0
7  6  6  6  6  0  6  6  6
8  1  0  0  4  5  3  2  1
schwarze Dame von D8 auf B8

   A  B  C  D  E  F  G  H
1 -1 -2 -3 -4 -5 -3 -2 -1
2 -6 -6  0 -6  0 -6 -6 -6
3  0  0  0  0 -6  0  0  0
4  0  0  0  0  0  0  0  0
5  0  0  0  0  6  0  0  0
6  0 -4  2  0  0  0  0  0
7  6  6  6  6  0  6  6  6
8  1

schwarzer König von E8 auf C8

   A  B  C  D  E  F  G  H
1 -1 -2 -3 -4 -5 -3 -2 -1
2 -6 -6  0  0  0 -6 -6 -6
3  0  0  0  0 -6  0  0  0
4  0  0  0 -6  0  0  0  0
5  0  0  0  0  6  0  0  0
6  0 -4  2  0  0  0  0  0
7  6  6  6  6  4  6  6  6
8  0  0  5  1  0  3  2  1
weißer Bauer von E2 auf E3

   A  B  C  D  E  F  G  H
1 -1 -2 -3 -4 -5 -3 -2 -1
2 -6 -6 -6 -6  0 -6 -6 -6
3  0  0  0  0 -6  0  0  0
4  0  0  0  0  0  0  0  0
5  0  0  0  0  0  0  0  0
6  0  0  0  0  0  0  0  0
7  6  6  6  6  6  6  6  6
8  1  2  3  4  5  3  2  1

weißer König geschlagen auf: E1


Schwarz hat gewonnen!


schwarzer Bauer von E7 auf E1



   A  B  C  D  E  F  G  H
1 -1 -2 -3 -4  4 -3 -2 -1
2 -6 -6 -6 -6  0 -6 -6 -6
3  0  0  0  0 -6  0  0  0
4  0  0  0  0  0  0  0  0
5  0  0  0  0  0  0  0  0
6  0  0  0  0  0  0  0  0
7  6  6  6  6  0  6  6  6
8  1  2  3  4  5  3  2  1
